In [1]:
import numpy as np
import pandas as pd

Linear regression generally have the form of $Y_{i} = \theta_{0} + \theta_{1} x_{1} + \theta_{2} x_{2} + ...$ <br>
There are several ways to find the coefficients of the regression: <br>
1. Linear Algebra: $\hat{\theta} = (X^{T}X)^{-1}X^{T}Y$ (When X is invertible) <br>
2. Gradient Descent: In this case, we need to write out the loss function and try to minimize the loss. <br>
$\hspace{30mm}$ $F(x)$ = Loss Function = SE = $ \sum^{n}_{i=1} (Y_{i} - \hat{Y_{i}})^{2}$ <br>

In [10]:
import numpy as np
import pandas as pd

class Linear_Regression():
    def __init__(self, alpha = 1e-10 , num_iter = 10000, early_stop = 1e-50, intercept = True, init_weight = None):  
        """
        Initializes Linear Regression class.
        ---
        Parameters: 
            alpha: Learning Rate, defaults to 1e-10.
            num_iter: Number of Iterations to update coefficient with training data.
            early_stop: Constant control early_stop.
            intercept: Bool, If we are going to fit a intercept, default True.
            init_weight: Matrix (n x 1), input init_weight for testing.
        """
        self.model_name = 'Linear Regression'
        self.alpha = alpha
        self.num_iter = num_iter
        self.early_stop = early_stop
        self.intercept = intercept
        self.init_weight = init_weight  ### For testing correctness.
        self.coef = None
        self.loss = []
        

    def __str__(self):
        return self.model_name
    

    def fit(self, X_train, y_train):
        """
        Saves datasets to our model, and performs gradient descent.
        ---
        Parameters:
            X_train: Matrix or 2-D array. Input feature matrix.
            y_train: Matrix or 2-D array. Input target value.
        """
        self.X = np.mat(X_train)
        self.y = np.reshape(y_train, (-1, 1))
        
        # adds column of all 1's to first column of X if intercept is True.
        if self.intercept:
            ones = np.ones((self.X.shape[0], 1))
            self.X = np.hstack((ones, self.X))
        
        if self.init_weight is not None:
            self.coef = self.init_weight
        else:
            # initializes coefficient with uniform from [-1, 1]
            self.coef = np.random.rand(self.X.shape[1]) * 2 - 1
            self.coef = np.reshape(self.coef, (-1, 1))

        # Call gradient_descent function to train.
        self.gradient_descent()
        

    def gradient(self):
        """
        Helper function to calculate the gradient of the cost function.
        Gradient: -2X^T(y - Xw)
        ---
        Returns the gradient of the cost function.
        """
        y_pred = np.dot(self.X, self.coef)
        y_pred = np.reshape(y_pred, (-1, 1))
        error = np.subtract(self.y, y_pred)

        # Return gradient of cost function
        return -2 * np.dot(self.X.T, error)
        

    def gradient_descent(self):
        """
        Performs gradient descent to find the model's optimal coefficients.
        Cost function: SE = \sum^{n}_{i=1}(Y_i - \hat{Y}_i)^2
        """
        pre_error = float('inf')
        for i in range(self.num_iter):
            # Calculate the gradient and temporary coefficient to compare
            temp_coef = self.coef - self.alpha * self.gradient()  

            # Calculate error for early stopping
            y_pred = np.dot(self.X, temp_coef)
            current_error = np.sum(np.power(y_pred - self.y, 2))

            ### This is the early stop, don't modify the following three lines.
            if (abs(pre_error - current_error) < self.early_stop) | (abs(abs(pre_error - current_error) / pre_error) < self.early_stop):
                self.coef = temp_coef
                return self
            
            # Update learning rate and coefficients based on error comparison
            if current_error < pre_error:
                self.coef = temp_coef
                # Increase learning rate if error decreases
                self.alpha *= 1.3 
            else:
                # Decrease learning rate if error increases
                self.alpha *= 0.9 

            # Update previous error for next iteration
            pre_error = current_error 
            # Add loss to loss list we create
            self.loss.append(current_error)

            if i % 1000 == 0:
                print('Iteration: ' + str(i))
                print('Coef:      ' + str(self.coef))
                print('Loss:      ' + str(current_error))
                print('Alpha:     ' + str(self.alpha)) 
    
        return self


    def ind_predict(self, x: list):
        """
        Predict the value based on its feature vector x.
        ---
        Parameters:
            x: Matrix, array or list. Input feature point.
        ---
        Returns a prediction of given data point.
        """
        x = np.matrix(x)
        if self.intercept:
            x = np.hstack(([1], x))
        return float(x * self.coef)
    

    def predict(self, X):
        """
            X is a matrix or 2-D numpy array, represnting testing instances. 
            Each testing instance is a feature vector. 
            ---
            Parameters:
                X: Matrix, array or list. Input feature point.
            ---
            Returns a prediction of the given data matrix.
        """
        X = np.matrix(X)

        # Adds ones if intercept is True.
        if self.intercept:
            ones = np.ones((X.shape[0], 1))
            X = np.hstack((ones, X))
            
        # Calls ind_predict for each value in the X matrix.
        predictions = [self.ind_predict(x) for x in X]
        return predictions



In [5]:
np.sum(np.power([[1],[2],[3]], 2))

14

In [14]:
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([3, 5, 7, 9, 11])

In [15]:
X

array([[1],
       [2],
       [3],
       [4],
       [5]])

In [16]:
y

array([ 3,  5,  7,  9, 11])

In [17]:
ones = np.ones((X.shape[0], 1))
X = np.hstack((ones, X))
X

array([[1., 1.],
       [1., 2.],
       [1., 3.],
       [1., 4.],
       [1., 5.]])

In [18]:
coef = np.random.rand(X.shape[1]) * 2 - 1
coef = np.reshape(coef, (-1, 1))
coef

array([[-0.146806  ],
       [ 0.72978767]])

In [19]:
y_pred = np.dot(X, coef)
y_pred = np.reshape(y_pred, (-1, 1))
y_pred

array([[0.58298167],
       [1.31276935],
       [2.04255702],
       [2.7723447 ],
       [3.50213237]])

In [20]:
y = np.reshape(y, (-1, 1))
y

array([[ 3],
       [ 5],
       [ 7],
       [ 9],
       [11]])

In [21]:
error = np.subtract(y, y_pred)
error

array([[2.41701833],
       [3.68723065],
       [4.95744298],
       [6.2276553 ],
       [7.49786763]])

In [22]:
-2 * np.dot(X.T, error )

array([[ -49.57442979],
       [-174.12753589]])

In [23]:
np.dot(X.T, error )

array([[24.78721489],
       [87.06376794]])

In [34]:
coef

array([[ 0.66172506],
       [-0.505454  ]])

In [11]:
clf = Linear_Regression(alpha = 1, num_iter = 10000000)
clf.fit(X,y)

/var/folders/lt/n_z12_lx2cb6n9qf36pmd3x40000gn/T/ipykernel_2209/4274744309.py:87: RuntimeWarning: invalid value encountered in double_scalars
  if (abs(pre_error - current_error) < self.early_stop) | (abs(abs(pre_error - current_error) / pre_error) < self.early_stop):
/var/folders/lt/n_z12_lx2cb6n9qf36pmd3x40000gn/T/ipykernel_2209/4274744309.py:84: RuntimeWarning: overflow encountered in power
  current_error = np.sum(np.power(y_pred - self.y, 2))
/Users/minchan/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


Iteration: 0
Coef:      [[ 67.19255825]
 [238.40403738]]
Loss:      3565130.703283351
Alpha:     1.3
Iteration: 1000
Coef:      [[-4.49621993e+152]
 [-1.62327806e+153]]
Loss:      inf
Alpha:     0.018801157180306803
Iteration: 2000
Coef:      [[4.31282153e+152]
 [1.55706542e+153]]
Loss:      1.5442095637279108e+308
Alpha:     0.02243034339904983
Iteration: 3000
Coef:      [[4.51697461e+152]
 [1.63077116e+153]]
Loss:      inf
Alpha:     0.018526203146264124
Iteration: 4000
Coef:      [[-4.49415875e+152]
 [-1.62253391e+153]]
Loss:      1.6767954355163624e+308
Alpha:     0.022102315004659825
Iteration: 5000
Coef:      [[-4.54482069e+152]
 [-1.64082448e+153]]
Loss:      inf
Alpha:     0.01825527012646606
Iteration: 6000
Coef:      [[4.58860375e+152]
 [1.65663155e+153]]
Loss:      inf
Alpha:     0.021779083800647574
Iteration: 7000
Coef:      [[3.92261973e+152]
 [1.41619019e+153]]
Loss:      inf
Alpha:     0.017988299316335894
Iteration: 8000
Coef:      [[-4.54355718e+152]
 [-1.64036831e+15

Iteration: 82000
Coef:      [[-4.27218463e+152]
 [-1.54239421e+153]]
Loss:      inf
Alpha:     0.017972580384828782
Iteration: 83000
Coef:      [[4.04203829e+152]
 [1.45930408e+153]]
Loss:      1.3563885389151521e+308
Alpha:     0.021441826475499928
Iteration: 84000
Coef:      [[3.9443533e+152]
 [1.4240367e+153]]
Loss:      inf
Alpha:     0.017709743718363383
Iteration: 85000
Coef:      [[-4.51152039e+152]
 [-1.62880201e+153]]
Loss:      inf
Alpha:     0.021128254463408228
Iteration: 86000
Coef:      [[-4.45901077e+152]
 [-1.60984437e+153]]
Loss:      inf
Alpha:     0.017450750857949147
Iteration: 87000
Coef:      [[4.00957935e+152]
 [1.44758537e+153]]
Loss:      1.3346914820052615e+308
Alpha:     0.02081926822701432
Iteration: 88000
Coef:      [[4.04184653e+152]
 [1.45923485e+153]]
Loss:      inf
Alpha:     0.017195545590557828
Iteration: 89000
Coef:      [[-4.36694638e+152]
 [-1.57660621e+153]]
Loss:      inf
Alpha:     0.020514800702492477
Iteration: 90000
Coef:      [[4.62907099e+1

Iteration: 152000
Coef:      [[3.97471598e+152]
 [1.43499858e+153]]
Loss:      1.3115820535977085e+308
Alpha:     0.022391159374442773
Iteration: 153000
Coef:      [[4.39722075e+152]
 [1.58753621e+153]]
Loss:      inf
Alpha:     0.018493839343934226
Iteration: 154000
Coef:      [[-4.15609361e+152]
 [-1.50048167e+153]]
Loss:      1.4340157018275579e+308
Alpha:     0.02206370401954875
Iteration: 155000
Coef:      [[-4.58026069e+152]
 [-1.65361944e+153]]
Loss:      inf
Alpha:     0.018223379622557257
Iteration: 156000
Coef:      [[4.48481912e+152]
 [1.61916201e+153]]
Loss:      1.6698333438339425e+308
Alpha:     0.02174103747472292
Iteration: 157000
Coef:      [[4.03115505e+152]
 [1.45537488e+153]]
Loss:      inf
Alpha:     0.01795687518918326
Iteration: 158000
Coef:      [[-3.85755238e+152]
 [-1.39269881e+153]]
Loss:      1.2353980965099738e+308
Alpha:     0.021423089706901002
Iteration: 159000
Coef:      [[-4.45205498e+152]
 [-1.60733311e+153]]
Loss:      inf
Alpha:     0.01769426820043

Iteration: 222000
Coef:      [[4.38883417e+152]
 [1.58450839e+153]]
Loss:      inf
Alpha:     0.01931265065821459
Iteration: 223000
Coef:      [[-4.40896683e+152]
 [-1.59177692e+153]]
Loss:      1.6138268179704605e+308
Alpha:     0.023040570431663722
Iteration: 224000
Coef:      [[-4.60073899e+152]
 [-1.66101276e+153]]
Loss:      inf
Alpha:     0.019030216382727828
Iteration: 225000
Coef:      [[4.35089508e+152]
 [1.57081118e+153]]
Loss:      1.5715944477124459e+308
Alpha:     0.02270361788528191
Iteration: 226000
Coef:      [[3.85050535e+152]
 [1.39015461e+153]]
Loss:      inf
Alpha:     0.018751912514888475
Iteration: 227000
Coef:      [[-4.31655985e+152]
 [-1.55841507e+153]]
Loss:      1.5468877403211695e+308
Alpha:     0.02237159303888265
Iteration: 228000
Coef:      [[-3.91662163e+152]
 [-1.41402468e+153]]
Loss:      inf
Alpha:     0.018477678650316437
Iteration: 229000
Coef:      [[3.95209527e+152]
 [1.42683179e+153]]
Loss:      1.2966957137956968e+308
Alpha:     0.02204442382823

...

Iteration: 9671000
Coef:      [[3.89184876e+152]
 [1.40508089e+153]]
Loss:      1.2574628836415569e+308
Alpha:     0.02094367455976789
Iteration: 9672000
Coef:      [[4.12985903e+152]
 [1.49101015e+153]]
Loss:      inf
Alpha:     0.017298298230241996
Iteration: 9673000
Coef:      [[-3.97699640e+152]
 [-1.43582189e+153]]
Loss:      1.3130874803274132e+308
Alpha:     0.020637387677920217
Iteration: 9674000
Coef:      [[-4.22270327e+152]
 [-1.52452987e+153]]
Loss:      inf
Alpha:     0.017045322478012277
Iteration: 9675000
Coef:      [[3.81597761e+152]
 [1.37768900e+153]]
Loss:      1.2089125920572932e+308
Alpha:     0.020335580031735072
Iteration: 9676000
Coef:      [[4.40367824e+152]
 [1.58986757e+153]]
Loss:      inf
Alpha:     0.016796046322723527
Iteration: 9677000
Coef:      [[-3.77945356e+152]
 [-1.36450266e+153]]
Loss:      1.1858814932841076e+308
Alpha:     0.02003818611546172
Iteration: 9678000
Coef:      [[-3.97224778e+152]
 [-1.43410749e+153]]
Loss:      inf
Alpha:     0.01655

Iteration: 9741000
Coef:      [[4.63122207e+152]
 [1.67201812e+153]]
Loss:      inf
Alpha:     0.018064177182973876
Iteration: 9742000
Coef:      [[-4.56733095e+152]
 [-1.64895139e+153]]
Loss:      inf
Alpha:     0.02155110419797963
Iteration: 9743000
Coef:      [[-4.48382088e+152]
 [-1.61880161e+153]]
Loss:      inf
Alpha:     0.017800000976132567
Iteration: 9744000
Coef:      [[4.08837509e+152]
 [1.47603313e+153]]
Loss:      1.387665308968942e+308
Alpha:     0.021235934074115385
Iteration: 9745000
Coef:      [[4.04087009e+152]
 [1.45888232e+153]]
Loss:      inf
Alpha:     0.01753968816520215
Iteration: 9746000
Coef:      [[-3.84386633e+152]
 [-1.38775771e+153]]
Loss:      1.2266476141549344e+308
Alpha:     0.02092537309723799
Iteration: 9747000
Coef:      [[-4.43587040e+152]
 [-1.60148996e+153]]
Loss:      inf
Alpha:     0.017283182250666027
Iteration: 9748000
Coef:      [[4.48344837e+152]
 [1.61866712e+153]]
Loss:      inf
Alpha:     0.020619353861732664
Iteration: 9749000
Coef:    

Iteration: 9811000
Coef:      [[4.48196610e+152]
 [1.61813197e+153]]
Loss:      inf
Alpha:     0.015580575240827147
Iteration: 9812000
Coef:      [[-4.18373767e+152]
 [-1.51046205e+153]]
Loss:      inf
Alpha:     0.0185880927250875
Iteration: 9813000
Coef:      [[4.10338430e+152]
 [1.48145193e+153]]
Loss:      1.3978727844859383e+308
Alpha:     0.02217615112509205
Iteration: 9814000
Coef:      [[3.75862550e+152]
 [1.35698307e+153]]
Loss:      inf
Alpha:     0.018316254612629455
Iteration: 9815000
Coef:      [[-4.53308534e+152]
 [-1.63658764e+153]]
Loss:      1.7059686841347728e+308
Alpha:     0.021851840118439117
Iteration: 9816000
Coef:      [[-3.96005279e+152]
 [-1.42970471e+153]]
Loss:      inf
Alpha:     0.018048391946198956
Iteration: 9817000
Coef:      [[4.04277454e+152]
 [1.45956989e+153]]
Loss:      1.3568827091438914e+308
Alpha:     0.02153227193791702
Iteration: 9818000
Coef:      [[4.07213357e+152]
 [1.47016943e+153]]
Loss:      inf
Alpha:     0.017784446587623418
Iteration:

Iteration: 9881000
Coef:      [[-3.82793550e+152]
 [-1.38200618e+153]]
Loss:      inf
Alpha:     0.019411077090374654
Iteration: 9882000
Coef:      [[-4.30964732e+152]
 [-1.55591943e+153]]
Loss:      inf
Alpha:     0.0160324588467651
Iteration: 9883000
Coef:      [[4.11710122e+152]
 [1.48640418e+153]]
Loss:      inf
Alpha:     0.019127203395796936
Iteration: 9884000
Coef:      [[-4.1096385e+152]
 [-1.4837099e+153]]
Loss:      1.4021371866786701e+308
Alpha:     0.022819326295542206
Iteration: 9885000
Coef:      [[-4.12292726e+152]
 [-1.48850756e+153]]
Loss:      inf
Alpha:     0.018847481159383866
Iteration: 9886000
Coef:      [[4.15059505e+152]
 [1.49849651e+153]]
Loss:      1.4302237741142474e+308
Alpha:     0.022485609292970287
Iteration: 9887000
Coef:      [[3.99901310e+152]
 [1.44377062e+153]]
Loss:      inf
Alpha:     0.018571849668906022
Iteration: 9888000
Coef:      [[-4.06583841e+152]
 [-1.46789667e+153]]
Loss:      1.3724087938709908e+308
Alpha:     0.022156772672770948
Iterat

Iteration: 9951000
Coef:      [[-4.35988568e+152]
 [-1.57405707e+153]]
Loss:      inf
Alpha:     0.016742292995062844
Iteration: 9952000
Coef:      [[4.23591894e+152]
 [1.52930114e+153]]
Loss:      inf
Alpha:     0.01997405678625562
Iteration: 9953000
Coef:      [[4.46417795e+152]
 [1.61170989e+153]]
Loss:      inf
Alpha:     0.016497448438210813
Iteration: 9954000
Coef:      [[-3.94095522e+152]
 [-1.42280988e+153]]
Loss:      inf
Alpha:     0.019681949899593834
Iteration: 9955000
Coef:      [[-4.39654106e+152]
 [-1.58729083e+153]]
Loss:      inf
Alpha:     0.016256184565141996
Iteration: 9956000
Coef:      [[4.52931836e+152]
 [1.63522764e+153]]
Loss:      inf
Alpha:     0.019394114875886458
Iteration: 9957000
Coef:      [[4.23192806e+152]
 [1.52786031e+153]]
Loss:      inf
Alpha:     0.016018449010810815
Iteration: 9958000
Coef:      [[-4.06310034e+152]
 [-1.46690814e+153]]
Loss:      inf
Alpha:     0.01911048924206661
Iteration: 9959000
Coef:      [[4.30296758e+152]
 [1.55350783e+153

In [7]:
def min_max_normaliz(lst):
    """
    Helper function for normalize for faster training.
    """
    maximum = np.max(lst)
    minimum = np.min(lst)

    return (lst - minimum) / (maximum - minimum)

### We generate some easy data for testing. We should fit a line with, $Y = 30 * X + 20$

In [25]:
X = np.array(np.mat(np.arange(1, 1000, 5)).T)
y = np.array((30 * X)).flatten() + 20

#### Do NOT modify the following line, just run it when you are done.  You can also try different initialization, you will notice different coef at the end.

In [12]:
clf = Linear_Regression(alpha = 1, num_iter = 10000000, init_weight= np.mat([15,25]).T)
clf.fit(X,y)

/Users/minchan/Data Science/ml-models/Linear_Regression.py:83: RuntimeWarning: invalid value encountered in double_scalars
  if (abs(pre_error - current_error) < self.early_stop) | (abs(abs(pre_error - current_error) / pre_error) < self.early_stop):
/Users/minchan/Data Science/ml-models/Linear_Regression.py:80: RuntimeWarning: overflow encountered in power
  current_error = np.sum(np.power(y_pred - self.y, 2))
/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


Iteration: 0
Coef:      [[-9.98985000e+05]
 [-6.64663975e+08]]
Loss:      2.931949629258539e+25
Alpha:     1.3
Iteration: 1000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     8.242644677309584e-25
Iteration: 2000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      1.7976931348623153e+308
Alpha:     9.83372187442067e-25
Iteration: 3000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     8.122101649905912e-25
Iteration: 4000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     9.689910433821821e-25
Iteration: 5000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     8.003321481635986e-25
Iteration: 6000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     9.548202136947282e-25
Iteration: 7000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     7.886278391893531e-25
Iteration: 8000
Coef:      [[-2.47244678e+147]
 [-1.645817

KeyboardInterrupt: 

####  As the number of iteration increase, you should notice the coeficient converges to [20, 30]. 
#### It maybe very slow update. Feel free to stop.

In [ ]:
clf.coef

In [ ]:
np.array(clf.predict(X))

#### Please try to normalize the X and fit again with normalized X. You should find something interesting. Also think about what you should do for predicting.

##### You can also try this with the wine dataset we use in HW1. Try fit this function to that dataset with same features. If you look closely to the updates of coefficients. What do you find? This could be mentioned in your report. 

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
url_Wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine = pd.read_csv(url_Wine, delimiter=';')
X = wine[['density','alcohol']]
y = wine.quality

In [ ]:
lr = LinearRegression()
lr.fit(X,y)
## Squared Error with sklearn.
sum((lr.predict(X) - y)**2)

#### You will notice different coefficients, but the loss is very close to each other like 805. In your report, briefly discuss this problem.

In [ ]:
clf = Linear_Regression(alpha = 1, num_iter = 5000000)
clf.fit(X,y)

In [ ]:
sum((clf.predict(X) - y)**2)